# Emotion Classification in short texts with BERT

Applying BERT to the problem of multiclass text classification. Our dataset consists of written dialogs, messages and short stories. Each dialog utterance/message is labeled with one of the five emotion categories: joy, anger, sadness, fear, neutral.

# Don't run from top，it will take about 10hours to retrain
Scroll down↓ and start from "RUN FROM HERE"

In [1]:
# install ktrain on Google Colab
# !pip3 install ktrain

^C


In [3]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

## 1. Import Data

In [4]:
data_train = pd.read_csv('data/data_train.csv', encoding='utf-8')
data_test = pd.read_csv('data/data_test.csv', encoding='utf-8')

X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


,Emotion,Text
0,neutral,There are tons of other paintings that I thin...
1,sadness,"Yet the dog had grown old and less capable , a..."
2,fear,When I get into the tube or the train without ...
3,fear,This last may be a source of considerable disq...
4,anger,She disliked the intimacy he showed towards so...
5,sadness,When my family heard that my Mother's cousin w...
6,joy,Finding out I am chosen to collect norms for C...
7,anger,A spokesperson said : ` Glen is furious that t...
8,neutral,Yes .
9,sadness,"When I see people with burns I feel sad, actua..."


In [5]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class
y_train = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]

## 2. Data preprocessing

* The text must be preprocessed in a specific way for use with BERT. This is accomplished by setting preprocess_mode to ‘bert’. The BERT model and vocabulary will be automatically downloaded

* BERT can handle a maximum length of 512, but let's use less to reduce memory and improve speed. 

In [6]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


## 2. Training and validation


Loading the pretrained BERT for text classification 

In [7]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 350


C:\Users\astolo\anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


Wrap it in a Learner object

In [8]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

Train the model. More about tuning learning rates [here](https://github.com/amaiya/ktrain/blob/master/tutorial-02-tuning-learning-rates.ipynb)

In [9]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
1323/1323 [==============================] - 8541s 6s/step - loss: 0.9253 - accuracy: 0.6492 - val_loss: 0.6046 - val_accuracy: 0.7922
Epoch 2/3
1323/1323 [==============================] - 7787s 6s/step - loss: 0.4581 - accuracy: 0.8452 - val_loss: 0.5455 - val_accuracy: 0.8164
Epoch 3/3
1323/1323 [==============================] - 7763s 6s/step - loss: 0.1992 - accuracy: 0.9375 - val_loss: 0.5664 - val_accuracy: 0.8261


Validation

In [10]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

107/107 [==============================] - 949s 9s/step
              precision    recall  f1-score   support

         joy       0.84      0.85      0.85       707
     sadness       0.79      0.82      0.81       676
        fear       0.88      0.85      0.86       679
       anger       0.81      0.78      0.79       693
     neutral       0.81      0.83      0.82       638

    accuracy                           0.83      3393
   macro avg       0.83      0.83      0.83      3393
weighted avg       0.83      0.83      0.83      3393



array([[600,  17,  14,  15,  61],
       [ 21, 555,  29,  48,  23],
       [ 24,  29, 578,  35,  13],
       [ 23,  72,  28, 541,  29],
       [ 43,  27,   9,  30, 529]], dtype=int64)

#### Testing with other inputs

In [11]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [12]:
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

1/1 [==============================] - 0s 378ms/step
predicted: sadness (0.42)


## 4. Saving Bert model


In [13]:
# let's save the predictor for later use
predictor.save("bert_model")

Done! to reload the predictor use: ktrain.load_predictor

# RUN FROM HERE
### It takes about 10 hours to retrain, so I recommend starting from here with pretrained model
### The code below should be finished within 5 mins

In [1]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [2]:
predictor = ktrain.load_predictor("bert_model")
message = 'Screw this stupid assignment!' # Message here!

prediction = predictor.predict(message)

print(message)
print('predicted emotion: ',prediction)

C:\Users\astolo\anaconda3\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


1/1 [==============================] - 4s 4s/step
Screw this stupid assignment!
predicted emotion:  anger
